In [ ]:
import os
import sys
import shutil
import numpy as np
import pandas as pd

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.utils.notebook_utils import store_metadata

In [ ]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results

In [ ]:
df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating", "timestamp"]
)


last_100_user_ids = df['userID'].drop_duplicates().tail(100)
df_test_last_100 = df[df['userID'].isin(last_100_user_ids)]
df = df[~df['userID'].isin(last_100_user_ids)]
print(df)
# Sort DataFrame by userID
df_test_last_100 = df_test_last_100.sort_values(by='userID')
df = df.sort_values(by='userID')

print(df_test_last_100)
#take info about the last 100 userID and store it in a new df, remove it from original df


100%|██████████| 4.81k/4.81k [00:00<00:00, 8.29kKB/s]


       userID  itemID  rating  timestamp
0         196     242     3.0  881250949
1         186     302     3.0  891717742
2          22     377     1.0  878887116
3         244      51     2.0  880606923
4         166     346     1.0  886397596
...       ...     ...     ...        ...
99994     378      78     3.0  880056976
99996     716     204     5.0  879795543
99997     276    1090     1.0  874795795
99998      13     225     2.0  882399156
99999      12     203     3.0  879959583

[89868 rows x 4 columns]
       userID  itemID  rating  timestamp
63173     820     264     3.0  887955180
98094     820     258     3.0  887954853
96069     820     358     1.0  887954972
96581     820     347     4.0  887954853
64415     820     301     2.0  887955046
...       ...     ...     ...        ...
82198     943     402     2.0  888639702
68024     943     232     4.0  888639867
99832     943     739     4.0  888639929
79678     943      55     5.0  888639118
93735     943      28     4.0  

In [ ]:
train, test = python_chrono_split(df, 0.75)

In [ ]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [ ]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [ ]:
train_file = "data/train.csv"
test_file = "data/test.csv"
leave_one_out_test_file = "data/leave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [ ]:
data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)

  0%|          | 0/843 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dis

In [ ]:
model = NCF(
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16, 8, 4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

model.load(neumf_dir='NeuMF_trained_model1')

model.user2id = data.user2id
model.item2id = data.item2id
model.id2user = data.id2user
model.id2item = data.id2item

predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]


predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


,userID,itemID,prediction
0,1.0,88.0,0.623180
1,1.0,149.0,0.032055
2,1.0,110.0,0.010221
3,1.0,101.0,0.465207
4,1.0,103.0,0.011850


In [ ]:
df_test_last_100.to_csv("data/test_last_100_user.csv", index=False)

# Get the last interaction of each user in df_test_last_100 and store it in a new DataFrame
df_last_interaction = df_test_last_100.groupby('userID').last().reset_index()

print(df_last_interaction)

# Save df_last_interaction to CSV
df_last_interaction.to_csv("data/last_interaction_test_last_100_user.csv", index=False)

df_test_last_100 = df_test_last_100[~df_test_last_100.set_index(['userID', 'itemID']).index.isin(df_last_interaction.set_index(['userID', 'itemID']).index)]
print(df_test_last_100)

    userID  itemID  rating  timestamp
0      820     302     5.0  887954906
1      822     408     5.0  891037291
2      824     319     2.0  877020927
3      841     358     1.0  889067348
4      845    1399     3.0  885409493
..     ...     ...     ...        ...
95     939     890     2.0  880260636
96     940    1401     1.0  885921371
97     941     300     4.0  875048495
98     942     607     5.0  891282931
99     943      28     4.0  875409978

[100 rows x 4 columns]
       userID  itemID  rating  timestamp
63173     820     264     3.0  887955180
98094     820     258     3.0  887954853
96069     820     358     1.0  887954972
96581     820     347     4.0  887954853
64415     820     301     2.0  887955046
...       ...     ...     ...        ...
88402     943     614     5.0  888639351
82198     943     402     2.0  888639702
68024     943     232     4.0  888639867
99832     943     739     4.0  888639929
79678     943      55     5.0  888639118

[10032 rows x 4 columns]


In [ ]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 6.950319639000099 seconds for prediction.


In [ ]:
all_predictions['prediction'] = all_predictions['prediction'] * 4 + 1
all_predictions.head()


,userID,itemID,timestamp,prediction
2,1,3,NaN,2.691304
4,1,5,NaN,1.687650
5,1,6,NaN,1.366647
8,1,9,NaN,3.090109
11,1,12,NaN,4.185830


In [ ]:
def calculate_hr_ndcg(model, data, top_k=TOP_K):
    ndcgs = []
    hit_ratio = []

    for b in data.test_loader():
        user_input, item_input, labels = b
        output = model.predict(user_input, item_input, is_list=True)
        output_df = pd.DataFrame({'prediction': output, 'itemID': item_input})
        top_k_predictions = output_df.sort_values(by='prediction', ascending=False).head(top_k)
        output = np.squeeze(output)
        rank = sum(output >= output[0])
        if rank <= top_k:
            ndcgs.append(1 / np.log(rank + 1))
            hit_ratio.append(1)
        else:
            ndcgs.append(0)
            hit_ratio.append(0)

    eval_ndcg = np.mean(ndcgs)
    eval_hr = np.mean(hit_ratio)
    return eval_hr, eval_ndcg

# Evaluate model
eval_hr, eval_ndcg = calculate_hr_ndcg(model, data)
print("HR:\t%f" % eval_hr)
print("NDCG:\t%f" % eval_ndcg)

HR:	0.525504
NDCG:	0.399501


In [ ]:
new_user_data123 = df_test_last_100[df_test_last_100['userID'] == 820]
print(new_user_data123)

       userID  itemID  rating  timestamp
63173     820     264     3.0  887955180
98094     820     258     3.0  887954853
96069     820     358     1.0  887954972
96581     820     347     4.0  887954853
64415     820     301     2.0  887955046
66235     820     343     4.0  887955241
66261     820     316     3.0  887955204
73594     820     313     5.0  887954934
84240     820     324     3.0  887955020
67714     820     288     5.0  887954934
61539     820     748     1.0  887955223
91431     820     286     4.0  887954853
83459     820     333     5.0  887954878
99627     820     895     2.0  887955046
99686     820     751     1.0  887955180
62813     820     315     3.0  887954828
59396     820     289     2.0  887955020
71070     820     271     2.0  887955020
58401     820     538     3.0  887954906
86090     820     328     2.0  887955079


In [ ]:
min_rating = new_user_data123['rating'].min()
max_rating = new_user_data123['rating'].max()

# Áp dụng công thức chuẩn hóa
new_user_data123['rating_normalized'] = (new_user_data123['rating'] - min_rating) / (max_rating - min_rating)

print(new_user_data123)

# Chuẩn hóa giá trị rating của df_last_interaction
min_rating_interaction = df_last_interaction['rating'].min()
max_rating_interaction = df_last_interaction['rating'].max()

# Áp dụng công thức chuẩn hóa
df_last_interaction['rating_normalized'] = (df_last_interaction['rating'] - min_rating_interaction) / (max_rating_interaction - min_rating_interaction)

print(df_last_interaction)


       userID  itemID  rating  timestamp  rating_normalized
63173     820     264     3.0  887955180               0.50
98094     820     258     3.0  887954853               0.50
96069     820     358     1.0  887954972               0.00
96581     820     347     4.0  887954853               0.75
64415     820     301     2.0  887955046               0.25
66235     820     343     4.0  887955241               0.75
66261     820     316     3.0  887955204               0.50
73594     820     313     5.0  887954934               1.00
84240     820     324     3.0  887955020               0.50
67714     820     288     5.0  887954934               1.00
61539     820     748     1.0  887955223               0.00
91431     820     286     4.0  887954853               0.75
83459     820     333     5.0  887954878               1.00
99627     820     895     2.0  887955046               0.25
99686     820     751     1.0  887955180               0.00
62813     820     315     3.0  887954828

<ipython-input-87-45e92370b736>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_user_data123['rating_normalized'] = (new_user_data123['rating'] - min_rating) / (max_rating - min_rating)


In [ ]:
def calculate_user_embedding(history, model):
    embeddings = []
    weights = []  # Danh sách trọng số (ratings)

    for _, row in history.iterrows():
        if row['itemID'] in model.item2id:  # Kiểm tra nếu itemID tồn tại trong từ điển
            item_idx = model.item2id[row['itemID']]  # Lấy index của item

            # Lấy embedding từ phần GMF
            item_embedding_gmf = model.sess.run(model.embedding_gmf_Q)[item_idx]
            # Lấy embedding từ phần MLP
            item_embedding_mlp = model.sess.run(model.embedding_mlp_Q)[item_idx]

            # Kết hợp embedding từ GMF và MLP
            item_embedding = np.concatenate([item_embedding_gmf, item_embedding_mlp])

            embeddings.append(item_embedding)
            weights.append(row['rating_normalized'])  # Sử dụng rating làm trọng số

    if embeddings:
        # Tính trung bình có trọng số
        user_embedding = np.average(embeddings, axis=0, weights=weights)
    else:
        user_embedding = np.zeros(model.embedding_gmf_Q.shape[1] + model.embedding_mlp_Q.shape[1])  # Trả về vector 0 nếu không có

    return user_embedding


# Tính vector đại diện người dùng mới
user_embedding = calculate_user_embedding(new_user_data123, model)
user_embedding


array([-0.47353279, -0.41109434,  0.36233143,  0.00133323,  0.10075541,
       -0.36085782,  0.19298048,  0.47663349,  0.42595135, -0.57143402,
       -0.4360064 ,  0.37232214])

In [ ]:
print("User Embedding from Weighted Pooling:", user_embedding)
print("Mean Value of Embedding:", user_embedding.mean())
print("Standard Deviation of Embedding:", user_embedding.std())


User Embedding from Weighted Pooling: [-0.47353279 -0.41109434  0.36233143  0.00133323  0.10075541 -0.36085782
  0.19298048  0.47663349  0.42595135 -0.57143402 -0.4360064   0.37232214]
Mean Value of Embedding: -0.026718154223329446
Standard Deviation of Embedding: 0.38269211920758595


In [ ]:
new_user_id = 820

if new_user_id not in model.user2id:
    new_user_index = len(model.user2id)  # Gán chỉ số mới
    model.user2id[new_user_id] = new_user_index
    model.id2user[new_user_index] = new_user_id

print(f"Updated user2id: {model.user2id}")

Updated user2id: OrderedDict([(1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24), (26, 25), (27, 26), (28, 27), (29, 28), (30, 29), (31, 30), (32, 31), (33, 32), (34, 33), (35, 34), (36, 35), (37, 36), (38, 37), (39, 38), (40, 39), (41, 40), (42, 41), (43, 42), (44, 43), (45, 44), (46, 45), (47, 46), (48, 47), (49, 48), (50, 49), (51, 50), (52, 51), (53, 52), (54, 53), (55, 54), (56, 55), (57, 56), (58, 57), (59, 58), (60, 59), (61, 60), (62, 61), (63, 62), (64, 63), (65, 64), (66, 65), (67, 66), (68, 67), (69, 68), (70, 69), (71, 70), (72, 71), (73, 72), (74, 73), (75, 74), (76, 75), (77, 76), (78, 77), (79, 78), (80, 79), (81, 80), (82, 81), (83, 82), (84, 83), (85, 84), (86, 85), (87, 86), (88, 87), (89, 88), (90, 89), (91, 90), (92, 91), (93, 92), (94, 93), (95, 94), (96, 95), (97, 96), (98, 97), (99, 98),

In [ ]:
def predict_ratings_for_new_user(user_id, user_embedding, model, train_data, top_k=10):
    unseen_items = set(model.id2item.keys()) - set(train_data[train_data['userID'] == user_id]['itemID'])
    print(f"Unseen items: {len(unseen_items)}")

    predictions = []
    for item_id in unseen_items:
        if item_id in model.item2id:  # Kiểm tra nếu item_id có trong item2id
            item_idx = model.item2id[item_id]
            # Lấy embedding từ phần GMF
            item_embedding_gmf = model.sess.run(model.embedding_gmf_Q)[item_idx]
            # Lấy embedding từ phần MLP
            item_embedding_mlp = model.sess.run(model.embedding_mlp_Q)[item_idx]

            # Kết hợp embedding từ GMF và MLP
            item_embedding = np.concatenate([item_embedding_gmf, item_embedding_mlp])

            # Tính toán điểm dự đoán
            prediction = np.dot(user_embedding, item_embedding)
            predictions.append({'itemID': item_id, 'prediction': prediction})

    if not predictions:
        print("No predictions generated. Please check unseen items and model.item2id mapping.")
        return pd.DataFrame(columns=['itemID', 'prediction'])

    # Tạo DataFrame từ danh sách dự đoán
    predictions_df = pd.DataFrame(predictions)
    # predictions_df['prediction'] = predictions_df['prediction'].clip(1, 5)

    # Sắp xếp theo điểm dự đoán và lấy top K
    top_k_predictions = predictions_df.sort_values(by='prediction', ascending=False).head(top_k+10)
    return top_k_predictions


# Dự đoán và lấy kết quả
predictions_df = predict_ratings_for_new_user(new_user_id, user_embedding, model, train)
predictions_df


Unseen items: 1593


,itemID,prediction
299,300,3.444967
744,748,2.972818
257,258,2.935705
244,245,2.885404
327,328,2.680750
325,326,2.456509
287,288,2.440919
293,294,2.429745
322,323,2.402106
312,313,2.366840


In [ ]:
def generate_and_check_predictions(new_user_id, model, top_k=TOP_K):
    if new_user_id not in model.user2id:
        new_user_index = len(model.user2id)  # Gán chỉ số mới
        model.user2id[new_user_id] = new_user_index
        model.id2user[new_user_index] = new_user_id

    # Generate new user data and embedding
    new_user_data = df_test_last_100[df_test_last_100['userID'] == new_user_id]

    min_rating = new_user_data['rating'].min()
    max_rating = new_user_data['rating'].max()
    new_user_data['rating_normalized'] = (new_user_data['rating'] - min_rating) / (max_rating - min_rating)

    user_embedding = calculate_user_embedding(new_user_data, model)

    # Predict ratings for new user
    predictions_df = predict_ratings_for_new_user(new_user_id, user_embedding, model, new_user_data, top_k=top_k)

    # Check if df_last_interaction is in the top-k predictions
    last_interaction = df_last_interaction[df_last_interaction['userID'] == new_user_id]
    if not last_interaction.empty:
        item_id = last_interaction.iloc[0]['itemID']
        if item_id in predictions_df['itemID'].values:
          return True
        else:
          return False
    else:
      return False

correct_recommendations = 0
for user_id in df_test_last_100['userID'].unique():
    if generate_and_check_predictions(user_id, model):
        correct_recommendations += 1

print(f"Number of correct recommendations: {correct_recommendations} out of 100")

<ipython-input-93-abcef309fa59>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_user_data['rating_normalized'] = (new_user_data['rating'] - min_rating) / (max_rating - min_rating)


Unseen items: 1573


<ipython-input-93-abcef309fa59>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_user_data['rating_normalized'] = (new_user_data['rating'] - min_rating) / (max_rating - min_rating)


Unseen items: 1569


<ipython-input-93-abcef309fa59>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_user_data['rating_normalized'] = (new_user_data['rating'] - min_rating) / (max_rating - min_rating)


...

Unseen items: 1572


<ipython-input-93-abcef309fa59>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_user_data['rating_normalized'] = (new_user_data['rating'] - min_rating) / (max_rating - min_rating)


Unseen items: 1515


<ipython-input-93-abcef309fa59>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_user_data['rating_normalized'] = (new_user_data['rating'] - min_rating) / (max_rating - min_rating)


Unseen items: 1426
Number of correct recommendations: 30 out of 100
